In [3]:
import json
import os
import requests
import sys
sys.path.insert(0, "..")

import gradio as gr
from openai import OpenAI

c:\02_practice\01_RecommandDelivery\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
SYSTEM_PROMPT = f"""당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.
실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)
추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.
당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.
"""


MESSAGES = [
    {
        'role': 'system',
        'content': SYSTEM_PROMPT
    }
]


TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "recommend",
            "description": "사용자 발화 기반으로 메뉴 추천 API를 호출합니다. 오로지 이 함수 결과로만 메뉴 추천되어야 합니다. 사용자 발화가 없는 경우 빈 리스트가 반환 될 수 있습니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query_text": {
                        "type": "string",
                        "description": "사용자 발화 텍스트 원문",
                    },
                },
                "required": ["query_text"],
                "additionalProperties": False,
            },
        }
    }
]


def recommend(query_text):
    url = "http://localhost:8000/recommend"
    response = requests.post(url, json={"query_text": query_text})
    return response.json()


def call_openai(prompt, temperature=0.0, model='gpt-4o-2024-08-06'):
    client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

    MESSAGES.append(
        {
            'role': 'user',
            'content': prompt
        }
    )
    print("그냥 메시지",MESSAGES)
    completion = client.chat.completions.create(
        model=model,
        messages=MESSAGES,
        tools=TOOLS
    )
    print('1차 completion',completion)
    if completion.choices[0].finish_reason == "tool_calls":
        tool_calls = completion.choices[0].message.tool_calls
        tool_name = tool_calls[0].function.name
        tool_args = tool_calls[0].function.arguments
        tool_id = tool_calls[0].id
        MESSAGES.append(
            {
                "role": "assistant",
                "content": None,
                "tool_calls": [
                    {
                        "id": tool_id,
                        "type": "function",
                        "function": {
                            "name": tool_name,
                            "arguments": tool_args
                        }
                    }
                ]
            }
        )
        print("completion.choices[0].finish_reason TRUE메시지",MESSAGES)
        print("tool_args ->",tool_args)
        tool_result = recommend(**json.loads(tool_args))
        MESSAGES.append(
            {
                "role": "tool",
                "content": json.dumps(tool_result, ensure_ascii=False),
                "tool_call_id": tool_id
            }
        )
        print("recommend메시지", MESSAGES)
        completion = client.chat.completions.create(
            model=model,
            messages=MESSAGES,
            tools=TOOLS
        )
        print(completion)
    MESSAGES.append(
        {
            "role": "assistant",
            "content": completion.choices[0].message.content
        }
    )
    print(completion)
    return completion.choices[0].message.content


In [27]:
call_openai('아니 다이어트가 필요해')

그냥 메시지 [{'role': 'system', 'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"}, {'role': 'user', 'content': '아니 다이어트가 필요해'}]
1차 completion ChatCompletion(id='chatcmpl-BWzelGqqzTx4B3AaHVIZfk7ET7bEM', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_NrZ3YDaFKBFdSyTc86ZoExfE', function=Function(arguments='{"query_text":"다이어트"}', name='recommend'), type='function')], annotations=[]))], created=1747203203, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_f5bdcc3276', usage=CompletionUsage(completion_tokens=18, prompt_tokens=229, total_toke

'다이어트를 위한 메뉴로 아래와 같은 옵션들이 있습니다. 참고해보세요:\n\n1. [광어 1p - 초밥에미친남자-구로점](https://www.yogiyo.co.kr/mobile/#/488332/)\n2. [생새우타다끼 1p - 초밥에미친남자-구로점](https://www.yogiyo.co.kr/mobile/#/488332/)\n3. [제로콜라（펩시）- 스시현-가산점](https://www.yogiyo.co.kr/mobile/#/469768/)\n4. [생굴 - 화정족발이야기국내산족발본점](https://www.yogiyo.co.kr/mobile/#/60509/)\n\n이 메뉴들은 상대적으로 열량이 낮은 옵션들입니다. 맛있고 건강하게 다이어트를 하시길 바랍니다!'

In [22]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '숙취해소가 필요해'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_rxERws9OZP7frAegysOakqeN',
    'type': 'function',
    'function': {'name': 'recommend', 'arguments': '{"query_text":"숙취해소"}'}}]},
 {'role': 'tool',
  'content': '[{"_id": "1359676_9", "score": 0.5843505905046147, "menu": "수박주스", "restaurant": "옐로우캔-독산직영점", "url": "https://www.yogiyo.co.kr/mobile/#/1359676/"}, {"_id": "1359676_19", "score": 0.5694937338886034, "menu": "[죽장인이 끓인] 명품죽", "restaurant": "옐로우캔-독산직영점", "url": "https://www.yogiyo.co.kr/mobile/#/1359676/"}, {"_id": "1359676_20", "score": 0.5118686358714425, "menu": "미숫가루", "restaurant": "옐로우캔-독산직영점", "u

In [33]:
import os

import numpy as np
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

KEYWORDS_BLACKLIST = ['리뷰', 'zㅣ쀼', 'ZI쀼', 'Zl쀼', '리쀼', '찜', '이벤트', '추가', '소스']
KEYWORDS_CONTEXT = [
    '해장', '숙취',
    '다이어트'
]

def get_embedding(text, model = 'text-embedding-3-small'):
    clint = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
    respone = clint.embeddings.create(
        input = text,
        model = model
    )
    print(respone)
    print(respone.data)
    return respone.data[0].embedding

In [34]:
get_embedding('안녕하세요')

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.002531265141442418, -0.06127675995230675, -0.008443817496299744, 0.031540773808956146, 0.031089577823877335, -0.04993239790201187, -0.059171177446842194, 0.03244316577911377, -0.014116000384092331, -0.06140567362308502, -0.020722804591059685, 0.006832401733845472, 0.009195811115205288, -0.020067494362592697, -0.011484021320939064, 0.035214800387620926, -0.04653768241405487, 0.004017795901745558, -0.0141482288017869, 0.027394063770771027, 0.05706559494137764, -0.017306603491306305, -0.031368888914585114, -0.020185666158795357, 0.04413130134344101, 0.06239400804042816, 0.056936681270599365, 0.0005176672129891813, 0.017166946083307266, -0.05358493700623512, 0.0314333438873291, -0.02758743427693844, -0.01743551529943943, 0.0011414192849770188, -0.002343266736716032, 0.02537442371249199, 0.02264576032757759, -0.04391644522547722, -0.011752590537071228, -0.034204982221126556, -0.013546633534133434, -0.018176767975091934, -0.00301334704272

[-0.002531265141442418,
 -0.06127675995230675,
 -0.008443817496299744,
 0.031540773808956146,
 0.031089577823877335,
 -0.04993239790201187,
 -0.059171177446842194,
 0.03244316577911377,
 -0.014116000384092331,
 -0.06140567362308502,
 -0.020722804591059685,
 0.006832401733845472,
 0.009195811115205288,
 -0.020067494362592697,
 -0.011484021320939064,
 0.035214800387620926,
 -0.04653768241405487,
 0.004017795901745558,
 -0.0141482288017869,
 0.027394063770771027,
 0.05706559494137764,
 -0.017306603491306305,
 -0.031368888914585114,
 -0.020185666158795357,
 0.04413130134344101,
 0.06239400804042816,
 0.056936681270599365,
 0.0005176672129891813,
 0.017166946083307266,
 -0.05358493700623512,
 0.0314333438873291,
 -0.02758743427693844,
 -0.01743551529943943,
 0.0011414192849770188,
 -0.002343266736716032,
 0.02537442371249199,
 0.02264576032757759,
 -0.04391644522547722,
 -0.011752590537071228,
 -0.034204982221126556,
 -0.013546633534133434,
 -0.018176767975091934,
 -0.00301334704272449,
 0.

In [1]:
import os

import numpy as np
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

KEYWORDS_BLACKLIST =  ['리뷰', 'zㅣ쀼', 'ZI쀼', 'Zl쀼', '리쀼', '찜', '이벤트', '추가', '소스']
KEYWORDS_CONTEXT = [
    '해장', '숙취',
    '다이어트'
]

def extract_keywords(reviews_text):
    keywords = []    
    for word in reviews_text.split():
        if any(keyword in word for keyword in KEYWORDS_CONTEXT):
            keywords.append(word)
    return keywords

In [5]:
extract_keywords("숙취 매뉴 추천 해줘")

['숙취']